In [2]:
import requests
import re
import glob
import io
import os
from bs4 import BeautifulSoup

In [2]:
#url = 'https://presidente.gob.mx/22-01-21-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador/'
url = 'https://presidente.gob.mx/secciones/version-estenografica/page/69/'
#url = 'https://resultados.as.com/resultados/futbol/primera/clasificacion'

headers = {'User-Agent':'Windowss13'}
page = requests.get(url, headers=headers)

f = open('conferencia_nueva.txt', 'w', encoding = 'utf-8')
f.write(page.text)
f.close()

In [ ]:

re.findall('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?',
           open('./conferencia_nueva.txt', 'r', encoding = 'utf-8').read())


In [62]:
url_i = 'https://presidente.gob.mx/secciones/version-estenografica/page/'
n_page = 116

for i in range(1, n_page + 1):
    page = requests.get(url_i + str(i) + '/', headers = headers) 
    f = open('./Page/' + str(i) + '.txt', 'w', encoding='utf-8')
    f.write(page.text)
    f.close()

In [ ]:
urls_conf = []

for pagina in glob.glob('./Page/*'):
    urls_conf += [path for protocol, domain, path in 
                  re.findall(r'(http|ftp|https):\/\/([\w\-_]+(?:(?:\.[\w\-_]+)+))([\w\-\.,@?^=%&:/~\+#]*[\w\-\@?^=%&/~\+#])?', 
                             open(pagina, 'r', encoding = 'utf-8').read())
                  if "estenografica-de-la" in path]
print(urls_conf)

*Estos dos archivos los eliminos porque me da un error que no comprendo y no logro arreglar, no hara tanta diferencia a la hora de analizar*

In [ ]:
urls_conf_set = list(set(urls_conf))

#El valor de la posicion 350 da problemas
urls_conf_set.pop(350)
urls_conf_set.pop(424)

In [ ]:
for path in urls_conf_set:    
    page = requests.get('https://presidente.gob.mx' + str(path), headers = headers) 
    f = open('./Estenograficas/' + path.replace("/", '-'), 'w', encoding='utf-8')
    f.write(page.text)
    f.close()

*Este archivo lo elimino porque me da un error que no comprendo y no logro arreglar*

In [ ]:
os.remove(glob.glob('./Estenograficas/*')[757])

In [88]:

for pagina in glob.glob('./Estenograficas/*'):
    soup = BeautifulSoup(open(pagina, 'r', encoding = 'utf-8').read(), 'lxml')
    open('./Estenograficas_Limpias/' + pagina.replace("./Estenograficas\\", ""), 'w', encoding='utf-8').write(soup.get_text())

In [74]:
from datetime import datetime

def date_normaliza(str_date):
    datetime_object = datetime.strptime(str_date[:6] + "20" + str_date[6:], '%d.%m.%Y')
    new_format = datetime_object.strftime('%Y-%m-%d')
    return new_format

for i, pagina in enumerate(glob.glob('./Estenograficas_Limpias/*')):
    
    f = open(pagina, "r", encoding='utf-8').read()
    
    if i == 211 or i == 821:
        fechas = re.findall('[0-9][0-9]\.[0-9][0-9][0-9][0-9]', f)[0]
        fechas = fechas[:5] + "." + fechas[5:]
    elif i == 960:
        fecha_temp = re.findall('[0-9][0-9]\.[0-9][0-9][0-9]\.[0-9][0-9]', f)[0]
        fechas = ''
        for i in range(9):
            if i != 4:
                fechas += fecha_temp[i]
        

    else:
        fechas = re.findall('[0-9][0-9]\.[0-9][0-9]\.[0-9][0-9]', f)[0]
    
    
    open('./Estenograficas_Limpias_por_Fecha/' + date_normaliza(fechas), 'w', encoding='utf-8').write(f)

